In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib tk

In [3]:
#df_a=pd.read_csv('resis_time_10_06_15_01_grAl3_4.csv')
#df_b=pd.read_csv('resis_time_10_06_16_55_grAl3_4.csv')
df_a=pd.read_csv('resis_time_10_07_08_41_grAl3_4.csv')
df_b=pd.read_csv('resis_time_10_07_10_52_grAl3_4.csv')

df_a.head()

,Date,Resistance,Temperature_mxc
0,08:41:07,-0.039246,0.028501
1,08:41:08,-0.039175,0.028501
2,08:41:09,-0.039251,0.028501
3,08:41:10,-0.039398,0.028501
4,08:41:12,-0.039572,0.028501


In [102]:
fig, ax = plt.subplots()
ax.plot(xC,yC,'o-')
plt.show()

In [95]:
## read data
off=42
xE, xC = df_a['Temperature_mxc'], df_b['Temperature_mxc']
yE, yC = 1000*df_a['Resistance']+off, 1000*df_b['Resistance']+off
# Find unique temperature values and their corresponding indices
unique_temperaturesE, temperature_indicesE = np.unique(xE, return_inverse=True)
unique_temperaturesC, temperature_indicesC = np.unique(xC, return_inverse=True)

# Calculate the mean and standard deviation of resistance for each unique temperature
mean_resistanceE = np.zeros_like(unique_temperaturesE, dtype=float)
std_deviation_resistanceE = np.zeros_like(unique_temperaturesE, dtype=float)
mean_resistanceC = np.zeros_like(unique_temperaturesC, dtype=float)
std_deviation_resistanceC = np.zeros_like(unique_temperaturesC, dtype=float)

for i, temp in enumerate(unique_temperaturesE):
    #print(i, temp)
    indices = np.where(temperature_indicesE == i)
    #print("indices es ",indices[0])
    mean_resistanceE[i] = np.mean(yE[indices[0]])
    std_deviation_resistanceE[i] = np.std(yE[indices[0]])

for i, temp in enumerate(unique_temperaturesC):
    indices = np.where(temperature_indicesC == i)
    mean_resistanceC[i] = np.mean(yC[indices[0]])
    std_deviation_resistanceC[i] = np.std(yC[indices[0]])

    
from scipy.special import erf
    
# Define the error function (also known as the Gaussian error function)
def error_function(temperature, A, Tc, width, offset):
    return A * (1 + erf((temperature - Tc) / (np.sqrt(2) * width))) / 2 + offset

# Initial guess for the fit parameters
initial_guess = [57.0,1.9, 0.02, 2.0]
initial_guess2 = [57.0,2.1, 0.02, 2.0]
from scipy.optimize import curve_fit

# Fit the data to the error function

# Remove NaN values from the data
valid_indices = np.isfinite(mean_resistanceC)
temperatureC = unique_temperaturesC[valid_indices]
resistanceC = mean_resistanceC[valid_indices]

valid_indices = np.isfinite(mean_resistanceE)
temperatureE = unique_temperaturesE[valid_indices]
resistanceE = mean_resistanceE[valid_indices]

params, covariance = curve_fit(error_function, temperatureC, resistanceC, p0=initial_guess)
params2, covariance2 = curve_fit(error_function, temperatureE, resistanceE, p0=initial_guess2)

# Extract the fitted parameters
A_fit, Tc_fit, width_fit, offset_fit = params#params

# Generate the fitted curve using the fitted parameters
fitted_curve = error_function(unique_temperaturesC, A_fit, Tc_fit, width_fit, offset_fit)

# Extract the fitted parameters
A_fit2, Tc_fit2, width_fit2, offset_fit2 = params2
fitted_curve2 = error_function(unique_temperaturesE, A_fit2, Tc_fit2, width_fit2, offset_fit2)

theo_temp = np.linspace(0, 4, 5000)
fitted_curve3 = error_function(theo_temp, A_fit, Tc_fit, width_fit, offset_fit)
fitted_curve4 = error_function(theo_temp, A_fit2, Tc_fit2, width_fit2, offset_fit2)

fig, ax = plt.subplots()
#ax.scatter(df_a['Temperature_mxc'], 1000*df_a['Resistance'], label='Calentamiento',s=20)
#ax.scatter(df_b['Temperature_mxc'], 1000*df_b['Resistance'],marker='s',s=10, label='Enfriamiento')
ax.errorbar(unique_temperaturesC, mean_resistanceC, yerr=std_deviation_resistanceC, label='Calentamiento', fmt='o')
ax.errorbar(unique_temperaturesE, mean_resistanceE, yerr=std_deviation_resistanceE, label='Enfriamiento', fmt='s')
#ax.plot(unique_temperaturesC, fitted_curve)
ax.plot(theo_temp, fitted_curve3, color='k', label='Ajuste C', lw=2)
ax.plot(theo_temp, fitted_curve4, color='g', label='Ajuste E', lw=2)

ax.set_xlabel('Temperatura (K)')
ax.set_ylabel(r'Resistencia (m$\Omega$)')
ax.tick_params(direction='in', length=4, width=1, colors='k', grid_color='r', grid_alpha=0.5)
ax.tick_params(right='true', top='true')
note=r'E: $T_c$ = '+str(round(Tc_fit,4))+'K'+r'$\pm$'+str(round(width_fit,4))+'K'
ax.text(0.0, 35, note, fontsize=12, color='green')
note=r'C: $T_c$ = '+str(round(Tc_fit2,4))+'K'+r'$\pm$'+str(round(width_fit2,4))+'K'
ax.text(2.2, 10, note, fontsize=12, color='k')
plt.title('Aluminio Granular')
##### promedio
average_Tc = (Tc_fit + Tc_fit2) / 2.0
average_width = (width_fit + width_fit2) / 2.0
# Generate the middle curve using the average parameters
middle_curve = error_function(theo_temp, (A_fit+A_fit2)/2.0, average_Tc, average_width, (offset_fit+offset_fit2)/2)

# Plot the middle curve
ax.plot(theo_temp, middle_curve,'--', color='b', label='Promedio', lw=2)
note=r'P: $T_c$ = '+str(round(average_Tc,4))+'K'+r'$\pm$'+str(round(average_width,4))+'K'
ax.text(2.2, 20, note, fontsize=12, color='blue')

print(Tc_fit, width_fit)
plt.legend()
plt.savefig('Tc_gral.png', dpi=300)

C:\Users\Quantum Circuits IB\.conda\envs\kelvin\lib\site-packages\scipy\optimize\_minpack_py.py:906: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


1.8790475616105586 0.013123143955110805
